# Run PESTPP-IES

In this notebook we will run PESTPP-IES in standard, basic mode and then use buildin functionality for resolving prior-data conflict and estimating total error covariance

In [ ]:
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.size']=12
import flopy
import pyemu
%matplotlib inline

## SUPER IMPORTANT: SET HOW MANY PARALLEL WORKERS TO USE

In [ ]:
num_workers = 5

In [ ]:
t_d = "template_history"
m_d = "master_ies"

### As usual, load the model and plot the domain just to remember why we are doing all this....

In [ ]:
m = flopy.modflow.Modflow.load("freyberg.nam",model_ws=t_d,check=False,forgive=False)

In [ ]:
# plot some model attributes
fig = plt.figure(figsize=(12,7))
ax = plt.subplot(111,aspect="equal")
mm = flopy.plot.PlotMapView(model=m)
mm.plot_grid()
mm.plot_ibound()
mm.plot_bc('SFR')
mm.plot_bc("GHB")
ax = mm.ax
#m.wel.stress_period_data.plot(ax=ax,mflay=2)

# plot obs locations
obs = pd.read_csv(os.path.join("..","base_model_files","obs_loc.csv"))
                  
obs_x = [m.sr.xcentergrid[r-1,c-1] for r,c in obs.loc[:,["row","col"]].values]
obs_y = [m.sr.ycentergrid[r-1,c-1] for r,c in obs.loc[:,["row","col"]].values]
ax.scatter(obs_x,obs_y,marker='.',label="water-level obs",s=80)

#plot names on the pumping well locations
wel_data = m.wel.stress_period_data[0]
wel_x = m.sr.xcentergrid[wel_data["i"],wel_data["j"]]
wel_y = m.sr.ycentergrid[wel_data["i"],wel_data["j"]]
for i,(x,y) in enumerate(zip(wel_x,wel_y)):
    ax.scatter([x],[y],color="red",marker="s",s=50)
    #ax.text(x,y,"{0}".format(i+1),ha="center",va="center")

ax.set_ylabel("y(m)")
ax.set_xlabel("x(m)")
plt.show()

### Run PESTPP-IES in original mode and post process

Load the existing pest control file and set some problem specific PESTPP-IES settings (these all have decent internal defaults, but thru some testing, we have found these speed up the process for the synthetic freyberg problem)

In [ ]:

pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
pst.pestpp_options["ies_num_reals"] = 50  # enough?
pst.pestpp_options["ies_par_en"] = "prior.jcb"
pst.pestpp_options["ies_bad_phi_sigma"] = 2.0
pst.pestpp_options["overdue_giveup_fac"] = 1.5
pst.pestpp_options["ies_save_rescov"] = True
pst.pestpp_options["ies_no_noise"] = True
pst.control_data.noptmax = 3



write the control file with the new PESTPP-IES specific settings

In [ ]:
pst.write(os.path.join(t_d,"freyberg_ies.pst"))

Run PESTPP-IES in parallel locally

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg_ies.pst",num_workers=num_workers,master_dir=m_d)

A cheap phi progress plot

In [ ]:
phi = pd.read_csv(os.path.join(m_d,"freyberg_ies.phi.actual.csv"),index_col=0)
phi.index = phi.total_runs
phi.iloc[:,6:].apply(np.log10).plot(legend=False,lw=0.5,color='k')
plt.ylabel('log \$Phi$')
plt.figure()
phi.iloc[-1,6:].hist()
plt.title('Final $\Phi$ Distribution');

That is a pretty amaz phi decrease for just a few model runs and 11K+ parameters

But, we are focused on making the forecasts of interest.  Since PESTPP-IES evaluates a prior parameter ensemble and we will treat the final iteration parameter ensemble as a posterier sample. So let's load corresponding output ("observation") ensembles and plot forecast prior and posterior histograms with "truth" value (red line)

In [ ]:
oe_pr = pd.read_csv(os.path.join(m_d,"freyberg_ies.0.obs.csv"),index_col=0)
oe_pt = pd.read_csv(os.path.join(m_d,"freyberg_ies.{0}.obs.csv".format(pst.control_data.noptmax)),index_col=0)
obs = pst.observation_data
fnames = pst.pestpp_options["forecasts"].split(",")
for forecast in fnames:
    ax = plt.subplot(111)
    oe_pr.loc[:,forecast].hist(ax=ax,color="0.5",alpha=0.5, label='prior')
    oe_pt.loc[:,forecast].hist(ax=ax,color="b",alpha=0.5, label='posterior')
    ax.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r", label='truth')
    ax.set_title(forecast)
    ax.legend(loc='upper right')
    plt.show()

Thoughts on this?  The prior covered the truth for all forecast but not so for the posterior.  This means we have incurred forecast-sensitive bias thru the parameter adjustment process.  #sad.  Let's how we did for observed vs simulated:

In [ ]:
nz_obs = pst.observation_data.loc[pst.nnz_obs_names,:].copy()
nz_obs.loc[:,"datetime"] = pd.to_datetime(nz_obs.obsnme.apply(lambda x: x.split("_")[-1]))
pst_base = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
oe_base = pd.read_csv(os.path.join(m_d,"freyberg_ies.obs+noise.csv"),index_col=0)
for nz_group in pst.nnz_obs_groups:
    nz_obs_group = nz_obs.loc[nz_obs.obgnme==nz_group,:]
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    
    [ax.plot(nz_obs_group.datetime,oe_pr.loc[r,nz_obs_group.obsnme],color="0.5",lw=0.1) for r in oe_pr.index]
    [ax.plot(nz_obs_group.datetime,oe_pt.loc[r,nz_obs_group.obsnme],color="b",lw=0.1,alpha=0.5) for r in oe_pt.index]
    ax.plot(nz_obs_group.datetime,nz_obs_group.obsval,"r-")
    #[ax.plot(nz_obs_group.datetime,oe_base.loc[r,nz_obs_group.obsnme],color="r",lw=0.1,alpha=0.5) for r in oe_base.index]
    mn = oe_base.loc[:,nz_obs_group.obsnme].min()
    mx = oe_base.loc[:,nz_obs_group.obsnme].max()
    
    ax.fill_between(nz_obs_group.datetime,mn,mx,fc="r",alpha=0.15)
    ax.set_title(nz_group)
    #vmin = min(nz_obs_group.obsval.min(),oe_pt.loc[:,nz_obs_group.obsnme].min().min())
    #vmax = max(nz_obs_group.obsval.max(),oe_pt.loc[:,nz_obs_group.obsnme].max().max())
    vmin = nz_obs_group.obsval.min() * 0.9
    vmax = nz_obs_group.obsval.max() * 1.1
    ax.set_ylim(vmin,vmax)
plt.show()

How can fit the observations so well and yet get the wrong "answer" for several of the foreacsts?  Here we see a very important outcome:  When you are using an imperfect model (compared to the truth), the link between a good fit of historic conditions and robust forecasts is broken: a good fit doesn't mean a good forecaster.

In [ ]:
pe_pr = pd.read_csv(os.path.join(m_d,"freyberg_ies.0.par.csv"),index_col=0)
pe_pt = pd.read_csv(os.path.join(m_d,"freyberg_ies.{0}.par.csv".format(pst.control_data.noptmax)),index_col=0)
par = pst.parameter_data
pdict = par.groupby("pargp").groups
pyemu.plot_utils.ensemble_helper({"0.5":pe_pr,"b":pe_pt},plot_cols=pdict)

In [ ]:
def plot_hk(real, _pe_pr, _pe_pt):
    # replace the par values in the control file
    _pst = pyemu.Pst(os.path.join(m_d,"freyberg.pst"))
    _pst.parameter_data.loc[:,"parval1"] = _pe_pr.loc[real,pst.par_names]
    # write the template files and run the array multiplier
    os.chdir(t_d)
    _pst.write_input_files()
    pyemu.helpers.apply_array_pars()
    os.chdir("..")
    
    hk_truth = np.log10(np.loadtxt(os.path.join('..','these_arent_the_files_youre_looking_for','hk_Layer_1.ref')))
    
    # load the arrays
    base_arr = np.log10(np.loadtxt(os.path.join(t_d,"arr_org","hk_Layer_1.ref")))
    pr_pp_arr = np.log10(np.loadtxt(os.path.join(t_d,"arr_mlt","hk0.dat_pp")))
    pr_gr_arr = np.log10(np.loadtxt(os.path.join(t_d,"arr_mlt","hk3.dat_gr")))
    pr_cn_arr = np.log10(np.loadtxt(os.path.join(t_d,"arr_mlt","hk6.dat_cn")))
    pr_in_arr = np.log10(np.loadtxt(os.path.join(t_d,"hk_Layer_1.ref")))
    pr_arrs = [base_arr,pr_cn_arr,pr_pp_arr,pr_gr_arr,pr_in_arr, hk_truth]
    
    # replace the par values in the control file
    _pst.parameter_data.loc[:,"parval1"] = _pe_pt.loc[real,pst.par_names]
    # write the template files and run the array multiplier
    os.chdir(t_d)
    _pst.write_input_files()
    pyemu.helpers.apply_array_pars()
    os.chdir("..")
    # load the arrays
    base_arr = np.log10(np.loadtxt(os.path.join(t_d,"arr_org","hk_Layer_1.ref")))
    pt_pp_arr = np.log10(np.loadtxt(os.path.join(t_d,"arr_mlt","hk0.dat_pp")))
    pt_gr_arr = np.log10(np.loadtxt(os.path.join(t_d,"arr_mlt","hk3.dat_gr")))
    pt_cn_arr = np.log10(np.loadtxt(os.path.join(t_d,"arr_mlt","hk6.dat_cn")))
    pt_in_arr = np.log10(np.loadtxt(os.path.join(t_d,"hk_Layer_1.ref")))
    pt_arrs = [base_arr,pt_cn_arr,pt_pp_arr,pt_gr_arr,pt_in_arr, hk_truth]
    
    
    # mask with ibound
    ib = m.bas6.ibound[0].array
    for i,arr in enumerate(pr_arrs):
        arr[ib==0] = np.NaN
    for i,arr in enumerate(pt_arrs):
        arr[ib==0] = np.NaN
    
    fig,axes = plt.subplots(2,6,figsize=(18,5))
    
    # work out the multiplier min and max
    vmin1 = min([np.nanmin(a) for a in pr_arrs[1:-1]])
    vmax1 = max([np.nanmax(a) for a in pr_arrs[1:-1]])
    vmin2 = min([np.nanmin(a) for a in pt_arrs[1:-1]])
    vmax2 = max([np.nanmax(a) for a in pt_arrs[1:-1]])
    vmin = min(vmin1,vmin2)
    vmax = min(vmax1,vmax2)
    
    in_vmin = min(hk_truth.ravel())
    in_vmax = max(hk_truth.ravel())
   
   
    labels = ["org","constant","pilot points","grid","input", "truth"]
    # mask with ibound
    # plot each array
    for i,(ax,arr,label) in enumerate(zip(axes[0,:],pr_arrs,labels)):
        if i >= len(pr_arrs)-2:
            cb = ax.imshow(arr,vmin=in_vmin,vmax=in_vmax)
        elif i == 0:
            cb = ax.imshow(arr)
        else:
            cb = ax.imshow(arr, vmin=vmin, vmax=vmax)
        if "truth" not in label:
            ax.set_title("prior "+label)
        else:
            ax.set_title(label)
        ax.set_yticks([])
        ax.set_xticks([])
        plt.colorbar(cb,ax=ax)
    for i,(ax,arr,label) in enumerate(zip(axes[1,:],pt_arrs,labels)):
        if i >= len(pr_arrs)-2:
            cb = ax.imshow(arr,vmin=in_vmin,vmax=in_vmax)
        elif i == 0:
            cb = ax.imshow(arr)
        else:
            cb = ax.imshow(arr, vmin=vmin, vmax=vmax)
        if "truth" not in label:
            ax.set_title("post "+label)
        else:
            ax.set_title(label)
        ax.set_yticks([])
        ax.set_xticks([])
        plt.colorbar(cb,ax=ax)
    plt.tight_layout()
    plt.show()

In [ ]:
pe_pt.index.values

In [ ]:
pe_pr.index.values

In [ ]:
plot_hk('base', pe_pr, pe_pt)

In [ ]:
plot_hk(pe_pt.index[0], pe_pr, pe_pt)

### PESTPP-IES with automatic prior-data conflict resolution

Prior-data conflict, in the simpliest sense, means that simulated outputs from the prior parameter ensemble don't "cover" the observed values (plus optional measurement noise).  If the outputs from using lots of parameters and conservative (wide) parameter ranges (from the Prior) don't cover the observed values, then that implies we will need extreme parameter values (or extreme combinations) to reproduce these observations - another word for extreme is baised. So we shouldnt attempt parameter adjustments in the presence of prior-data conflict.  The easies way to deal with this is to simply not use conflicted observations for parameter adjustment calculations...PESTPP-IES will do this automatically for you:

In [ ]:
pst.pestpp_options["ies_drop_conflicts"] = True
pst.pestpp_options["ies_pdc_sigma_distance"] = 2.0
pst.pestpp_options["ies_no_noise"] = True
pst.write(os.path.join(t_d,"freyberg_ies.pst"))

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg_ies.pst",num_workers=num_workers,master_dir=m_d)

In [ ]:
oe_pr_last = oe_pr.copy()
oe_pt_last = oe_pt.copy()
oe_pr = pd.read_csv(os.path.join(m_d,"freyberg_ies.0.obs.csv"),index_col=0)
oe_pt = pd.read_csv(os.path.join(m_d,"freyberg_ies.{0}.obs.csv".format(pst.control_data.noptmax)),index_col=0)
obs = pst.observation_data
fnames = pst.pestpp_options["forecasts"].split(",")
for forecast in fnames:
    fig,[ax,ax_last] = plt.subplots(1,2,figsize=(10,5))
    oe_pr.loc[:,forecast].hist(ax=ax,color="0.5",alpha=0.5, label='prior',density=True)
    oe_pt.loc[:,forecast].hist(ax=ax,color="b",alpha=0.5, label='posterior',density=True)
    ax.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r", label='truth')
    ax.set_title(forecast)
    ax.legend(loc='upper right')
    
    oe_pr_last.loc[:,forecast].hist(ax=ax_last,color="0.5",alpha=0.5, label='prior',density=True)
    oe_pt_last.loc[:,forecast].hist(ax=ax_last,color="b",alpha=0.5, label='posterior',density=True)
    ax_last.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r", label='truth')
    ax_last.set_title("last " + forecast)
    ax.legend(loc='upper right')
    plt.show()

In [ ]:
rw_obs = pd.read_csv(os.path.join(m_d,"freyberg_ies.adjusted.obs_data.csv"),index_col=0)
rw_obs = rw_obs.weight.to_dict()
nz_obs = pst.observation_data.loc[pst.nnz_obs_names,:].copy()
nz_obs.loc[:,"datetime"] = pd.to_datetime(nz_obs.obsnme.apply(lambda x: x.split("_")[-1]))
pst_base = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
oe_base = pd.read_csv(os.path.join(m_d,"freyberg_ies.obs+noise.csv"),index_col=0)
for nz_group in pst.nnz_obs_groups:
    nz_obs_group = nz_obs.loc[nz_obs.obgnme==nz_group,:]
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    ax.plot(nz_obs_group.datetime,nz_obs_group.obsval,"r-")
    pdc_obs = nz_obs_group.loc[nz_obs_group.obsnme.apply(lambda x: rw_obs[x]==0),:]
   
    ax.scatter(pdc_obs.datetime,pdc_obs.obsval,marker='.',s=50,zorder=20,color='k',ls='-',lw=2,fc="k")
    [ax.plot(nz_obs_group.datetime,oe_pr.loc[r,nz_obs_group.obsnme],color="0.5",lw=0.1) for r in oe_pr.index]
    [ax.plot(nz_obs_group.datetime,oe_pt.loc[r,nz_obs_group.obsnme],color="b",lw=0.1,alpha=0.5) for r in oe_pt.index]
    #[ax.plot(nz_obs_group.datetime,oe_base.loc[r,nz_obs_group.obsnme],color="r",lw=0.1,alpha=0.5) for r in oe_base.index]
    mn = oe_base.loc[:,nz_obs_group.obsnme].min()
    mx = oe_base.loc[:,nz_obs_group.obsnme].max()
    
    ax.fill_between(nz_obs_group.datetime,mn,mx,fc="r",alpha=0.15)
    ax.set_title(nz_group)
    #vmin = min(nz_obs_group.obsval.min(),oe_pt.loc[:,nz_obs_group.obsnme].min().min())
    #vmax = max(nz_obs_group.obsval.max(),oe_pt.loc[:,nz_obs_group.obsnme].max().max())
    vmin = nz_obs_group.obsval.min() * 0.9
    vmax = nz_obs_group.obsval.max() * 1.1
    ax.set_ylim(vmin,vmax)
plt.show()

In [ ]:
pe_pr_pdr = pd.read_csv(os.path.join(m_d,"freyberg_ies.0.par.csv"),index_col=0)
pe_pt_pdr = pd.read_csv(os.path.join(m_d,"freyberg_ies.{0}.par.csv".format(pst.control_data.noptmax)),index_col=0)

In [ ]:
pe_pr_pdr.loc[pe_pr_pdr.index[0],:]

In [ ]:
plot_hk('base', pe_pr_pdr, pe_pt_pdr)

In [ ]:
plot_hk(pe_pt_pdr.index[0], pe_pr_pdr, pe_pt_pdr)

### PESTPP-IES in a total error covariance workflow

The concept of subjective weighting implies that the covariance measurement noise is itself uncertain (a hyper parameter).  Recently, it has been shown that this covariance matrix can be "estimated" in a outer iteration loop.  So let's do that!  First, lets look at the final (e.g. posterior) residual covariance matrix from the last run:

In [ ]:
res_cov_to_mod_file = os.path.join(m_d,'freyberg_ies_shrunk_res_to_mod.csv')
if not os.path.exists(res_cov_to_mod_file):
    res_cov_file = os.path.join(m_d,"freyberg_ies.{0}.shrunk_res.cov".format(pst.control_data.noptmax))
    assert os.path.exists(res_cov_file)
    shutil.copy2(res_cov_file, res_cov_to_mod_file)
res_cov = pyemu.Cov.from_ascii(res_cov_to_mod_file)
x = res_cov.to_pearson().x.copy()
x[np.abs(x) < 0.2] = np.NaN
x[x==1.0] = np.NaN

fig,ax = plt.subplots(1,1,figsize=(10,10))
cb = ax.imshow(x,cmap="plasma")
plt.colorbar(cb)

That would make a nice bohemain rug pattern!  Seriously tho, we see lots of correlation between residuals...so much for the "independence" assumption..

So what should we do?  Well, let's feed that covariance matrix to PESTPP-IES for the next run (an "outer" iteration).  During this run, the noise realizations that are paired with each parameter realization for the calculation of measurement phi will be drawn from this covariance matrix.  Additionally, the weights for non-zero weighted observations maybe lowered if the variance on the diagaonal of this matrix implies lower a weight (weights will never be increased).  In this way, PESTPP-IES is given information about how well it could (or couldn't) fit the observations last time.  In practice, this will keep PESTPP-IES from fitting the obseravtions as well, but more importantly, this helps prevent bias arising from irreducible residual and ultimately, leads to less biased and more covservative forecast estimates. 

Now we need to tell PESTPP-IES to use this covariance matirx and also change some options to be compatible with this mode of operation:

In [ ]:
minvar = ((1./obs.loc[res_cov.names,"weight"])**2).min()
shrink = np.zeros(res_cov.shape)
np.fill_diagonal(shrink,minvar)
lamb = 2. / (oe_pt.shape[0] + 1)
lamb = 0.2
print(lamb)
shrunk = (lamb * shrink) + ((1.-lamb) * res_cov.x)
shrunk = pyemu.Cov(x=shrunk,names=res_cov.names)
shrunk.to_ascii(os.path.join(t_d,"shrunk_obs.cov"))
x = shrunk.to_pearson().x.copy()
x[x==0.0] = np.NaN
plt.imshow(x,cmap="plasma")

In [ ]:
pst.pestpp_options["ies_drop_conflicts"] = False
pst.pestpp_options["ies_pdc_sigma_distance"] = 2.0
pst.pestpp_options["ies_no_noise"] = False
#res_cov.to_ascii(os.path.join(t_d,"shrunk_obs.cov"))
pst.pestpp_options["obscov"] = "shrunk_obs.cov"
pst.pestpp_options["ies_group_draws"] = False
pst.write(os.path.join(t_d,"freyberg_ies.pst"))

Run it!

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg_ies.pst",num_workers=num_workers,master_dir=m_d)

In [ ]:
oe_pr_last = oe_pr.copy()
oe_pt_last = oe_pt.copy()
oe_pr = pd.read_csv(os.path.join(m_d,"freyberg_ies.0.obs.csv"),index_col=0)
oe_pt = pd.read_csv(os.path.join(m_d,"freyberg_ies.{0}.obs.csv".format(pst.control_data.noptmax)),index_col=0)
obs = pst.observation_data
fnames = pst.pestpp_options["forecasts"].split(",")
for forecast in fnames:
    fig,[ax,ax_last] = plt.subplots(1,2,figsize=(10,5))
    oe_pr.loc[:,forecast].hist(ax=ax,color="0.5",alpha=0.5, label='prior')
    oe_pt.loc[:,forecast].hist(ax=ax,color="b",alpha=0.5, label='posterior')
    ax.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r", label='truth')
    ax.set_title(forecast)
    ax.legend(loc='upper right')
    
    oe_pr_last.loc[:,forecast].hist(ax=ax_last,color="0.5",alpha=0.5, label='prior')
    oe_pt_last.loc[:,forecast].hist(ax=ax_last,color="b",alpha=0.5, label='posterior')
    ax_last.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r", label='truth')
    ax_last.set_title("last " + forecast)
    ax.legend(loc='upper right')
    plt.show()

Ok, now that is much better!  We are bracketing the truth for all forecasts - that is the first time so far.  Sweet as!  But let's see the price these unbiased and conservative foreacasts:

In [ ]:
[f for f in os.listdir(m_d) if f.startswith("freyberg_ies") and f.endswith(".csv")]

In [ ]:
nz_obs = pst.observation_data.loc[pst.nnz_obs_names,:].copy()
nz_obs.loc[:,"datetime"] = pd.to_datetime(nz_obs.obsnme.apply(lambda x: x.split("_")[-1]))
pst_base = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
oe_base = pd.read_csv(os.path.join(m_d,"freyberg_ies.obs+noise.csv"),index_col=0)
for nz_group in pst.nnz_obs_groups:
    nz_obs_group = nz_obs.loc[nz_obs.obgnme==nz_group,:]
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    ax.plot(nz_obs_group.datetime,nz_obs_group.obsval,"r-")
    pdc_obs = nz_obs_group.loc[nz_obs_group.obsnme.apply(lambda x: rw_obs[x]==0),:]
   
    ax.scatter(pdc_obs.datetime,pdc_obs.obsval,marker='.',s=50,zorder=20,color='k',ls='-',lw=2,fc="k")
    #[ax.plot(nz_obs_group.datetime,oe_pr.loc[r,nz_obs_group.obsnme],color="0.5",lw=0.1) for r in oe_pr.index]
    [ax.plot(nz_obs_group.datetime,oe_pt.loc[r,nz_obs_group.obsnme],color="b",lw=0.1,alpha=0.75) for r in oe_pt.index]
    #[ax.plot(nz_obs_group.datetime,oe_base.loc[r,nz_obs_group.obsnme],color="r",lw=0.1,alpha=0.5) for r in oe_base.index]
    mn = oe_base.loc[:,nz_obs_group.obsnme].min()
    mx = oe_base.loc[:,nz_obs_group.obsnme].max()
    
    #ax.fill_between(nz_obs_group.datetime,mn,mx,fc="r",alpha=0.15)
    ax.set_title(nz_group)
    #vmin = min(nz_obs_group.obsval.min(),oe_pt.loc[:,nz_obs_group.obsnme].min().min())
    #vmax = max(nz_obs_group.obsval.max(),oe_pt.loc[:,nz_obs_group.obsnme].max().max())
    vmin = nz_obs_group.obsval.min() * 0.9
    vmax = nz_obs_group.obsval.max() * 1.1
    ax.set_ylim(vmin,vmax)
plt.show()

Who is comfortable with this situation?  We have shown that a good fit to historic conditions doesn't mean robust forecast estimates.  But in all cases the prior ensemble covers the forecasts, implying that somewhere between the prior and well-fit posterior is the optimal location for robust forecasting (optimal meaning conservative variance and no/little bias).  From a theoritical standpoint, using the posterior residual covariance matrix in an outer iteration scheme is one way to find (more) robust forecast estimates, especially when a model needs to make a range of forecasts (like regional models?)

In [ ]:
pe_pr_tec = pd.read_csv(os.path.join(m_d,"freyberg_ies.0.par.csv"),index_col=0)
pe_pt_tec = pd.read_csv(os.path.join(m_d,"freyberg_ies.{0}.par.csv".format(pst.control_data.noptmax)),index_col=0)


In [ ]:
plot_hk('base',pe_pr_tec, pe_pt_tec)

In [ ]:
plot_hk(pe_pt_tec.index[0],pe_pr_tec, pe_pt_tec)